Dataproc to MSSQL

export NUMWORKERS=2

export WORKER=n1-standard-8

export MASTER=n1-standard-8

gcloud dataproc clusters create aekanun-datascience-1 - zone asia-east1-a - master-machine-type=$MASTER - worker-machine-type=$WORKER - num-workers $NUMWORKERS - scopes cloud-platform - bucket 'oct22–23' - project bigdatainpractice1 - region asia-east1 - properties spark:spark.jars.packages=com.microsoft.azure:spark-mssql-connector:1.0.2 - properties spark:spark.driver.maxResultSize=50g - properties spark:spark.driver.memory=58g - properties spark:spark.executor-memory=1g - max-age 240m - image-version=1.4-ubuntu18 - enable-component-gateway - optional-components=JUPYTER,ANACONDA - properties 'spark:spark.yarn.executor.memoryOverhead = 8200' - properties dataproc:dataproc.conscrypt.provider.enable=false

In [10]:
server_name = "jdbc:sqlserver://34.72.0.181"
database_name = "inclassdb"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "spark_flightdetail"
username = "SA"
password = "Passw0rd123456" # Please specify password here

In [2]:
! rm -rf *.csv

In [3]:
! wget https://storage.googleapis.com/6oct/part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv

--2022-10-19 13:30:51--  https://storage.googleapis.com/6oct/part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 64.233.187.128, 74.125.204.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7055298 (6.7M) [text/csv]
Saving to: ‘part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv’

part-00000-d6b74527 100%[===================>]   6.73M  --.-KB/s    in 0.05s   

2022-10-19 13:30:51 (127 MB/s) - ‘part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv’ saved [7055298/7055298]



In [6]:
! hdfs dfs -rm /part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv

Deleted /part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv


In [7]:
! hdfs dfs -put *.csv /

In [8]:
! hdfs dfs -ls /

Found 5 items
drwxrwxrwt   - mapred hadoop          0 2022-10-19 12:24 /hadoop
-rw-r--r--   2 root   hadoop    7055298 2022-10-19 13:31 /part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv
drwxrwxrwt   - mapred hadoop          0 2022-10-19 12:24 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2022-10-19 12:25 /user
drwx-wx-wx   - hive   hadoop          0 2022-10-19 12:24 /var


In [9]:
raw_df = spark.read.option('header','true').option('inferSchema','true')\
.csv('/part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv')

In [12]:
raw_df.write.mode('overwrite').format("jdbc")\
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").save()

In [13]:
! wc -l part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv

70322 part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv


In [14]:
! head -3 part-00000-d6b74527-fccf-4068-951f-9888a8aa9784-c000.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,1326,1230,1559,1530,WN,1056,N459WN,153,180,143,29,56,ISP,PBI,1052,5,5,0,"",0,0,0,0,0,29
2008,1,3,4,1720,1700,41,45,WN,1843,N484WN,261,285,244,-4,20,LAS,PHL,2176,4,13,0,"",0,NA,NA,NA,NA,NA


In [15]:
read_result_df = spark.read.format("jdbc")\
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password)\
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()

In [16]:
read_result_df.count()

70321

In [17]:
read_result_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Car

In [19]:
read_result_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Year,70321,2008.0,0.0,2008,2008
Month,70321,6.3931826908036005,3.409611931530387,1,12
DayofMonth,70321,15.665249356522233,8.781878358991534,1,31
DayOfWeek,70321,3.9198674649109084,1.9837560369885714,1,7
DepTime,70321,1332.110481052311,478.1684921260477,1,NA
CRSDepTime,70321,1324.5548982523,464.0525170092195,1,2359
ArrTime,70321,1477.3613194929053,505.7453617691981,1,NA
CRSArrTime,70321,1492.5435787318156,482.23720240001785,1,2400
UniqueCarrier,70321,None,None,9E,YV


In [18]:
raw_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
Year,70321,2008.0,0.0,2008,2008
Month,70321,6.3931826908036005,3.409611931530401,1,12
DayofMonth,70321,15.665249356522233,8.78187835899152,1,31
DayOfWeek,70321,3.9198674649109084,1.9837560369885665,1,7
DepTime,70321,1332.110481052311,478.16849212604683,1,NA
CRSDepTime,70321,1324.5548982523,464.0525170092238,1,2359
ArrTime,70321,1477.3613194929053,505.74536176919895,1,NA
CRSArrTime,70321,1492.5435787318156,482.23720240002035,1,2400
UniqueCarrier,70321,None,None,9E,YV
